## Modelling text through character-level bigrams

In [3]:
names = [line.rstrip() for line in open("../data/streets_zh.txt")]
names[:5]

['Aargauerstrasse',
 'Abeggweg',
 'Abendweg',
 'Ackermannstrasse',
 'Ackersteinstrasse']

In [4]:
# Add $ to mark beginning and ending of names
for i in range(len(names)):
    names[i] = "$" + names[i] + "$"

In [5]:
chars = set()
bigrams = set()
freq = dict()
for street in names:
    for pair in zip(street, street[1:]):
            chars.add(pair[0])
            bigrams.add(pair)
            freq[pair] = 1+freq.get(pair, 0)

In [6]:
freq_per_char = {i : dict() for i in chars}
for bigram, f in freq.items():
    i, j = bigram
    freq_per_char[i][j] = f

In [17]:
from random import choices

for _ in range(100):
    gen = ["$"]
    for _ in range(30):
        c = gen[-1]
        samples = list(freq_per_char[c].keys())
        p_samples = list(freq_per_char[c].values())
        next_c = choices(samples, p_samples)[0]
        gen.append(next_c)
        if next_c=="$":
            break
    print("".join(gen))
        

$Mauf
$Ziellchtrasersswilistrahisster
$Bi
$Ranweldwersseneg
$Imgastrasckhg
$Kaswanstranbsldenpphmaseralwe
$Eke
$Je
$Be
$Herasenge
$Juratrasstrd-Schrtran
$Alssstrasenebe
$Häbersttghe
$Re
$Gikoffweg-Schraseg
$Eg
$Niv-Simpin-We
$Are
$Eykelalentzlzelle
$Gise
$Le
$Heple
$Antrblllastholba-Bof
$Strolmaveieg
$Basstraipole
$Werrastrlichwinaastzwe
$Latretwie
$Kägänasenrunnstrassse
$Tangse de
$Frastrasttraseindwehodie
$Welhonasegaunssegeginzwase
$Idetrade
$Masselzwe
$Aligasse
$Wastrase
$Spladeg
$Brasenzend
$Reg
$Stratreigipenrast.-Brachatryrt
$Lastache
$Dedälbieg
$Virckkolatterachoqussssssedig
$Haselalnhfrng
$Maveim Kellseligrtenwiliastzhb
$Pünraserchasseg
$Bo-Strasraistrase Höneltrhasst
$He
$Juchseasssssteggge
$Fraite
$Enassse
$Em Begplwigraranstteng
$Sinee
$Myepltr-Flstrrdensese
$Ze
$Kidorastrofeusequhasigastrasen
$Sikeriserobe
$Kaustebertrasstintrigie
$Waseuchne
$Hode
$Splastegeg
$Graserarma-Flratrffe
$Ristzranlarauleig
$Be-Lasterase
$Höbüte
$Batraurdogeg
$Dihostwimfussenamürsse
$Binotrollase


In [27]:
sorted(list(freq.items()), key= lambda x: x[1])[-5:]

[(('r', 'a'), 1189),
 (('e', '$'), 1283),
 (('a', 's'), 1309),
 (('s', 's'), 1359),
 (('s', 'e'), 1366)]

## Use a smarter way to tokenize 

In [134]:
#In every iteration step (Byte-pair encoding): 
# 1. find most commmon pair
# 2. merge them and add to vocabulary
# 3. get counts on new vocabulary

new_tokens = set()
n_iterations = 12
names2 = names.copy()
vocab2 = set()
bigrams2 = set()
freq2 = freq.copy()

for n in range(n_iterations):
    #1
    new_token, _ = sorted(list(freq2.items()), key= lambda x: x[1])[-1]
    print(f"{new_token=}")
    #2
    new_tokens.add("".join(new_token))

    # need to treat words in wordlist as list of chars so we can handle merged tokens
    new_names = names.copy()
    temp_list = []
    for i, street in enumerate(names2):
        temp = []
        skip = False
        for i in range(len(street)-1):
            if skip:
                skip = False
                continue
            if new_token == (street[i], street[i+1]):
                temp.append("".join(new_token))
                skip = True
            else:
                temp.append(street[i])
        temp_list.append(temp+[street[-1]] if temp[-1][-1]!="$" else temp)
    names2 = temp_list
    
    # create count of tokens
    freq2 = dict()
    for street in names2:
        for pair in zip(street, street[1:]):
                vocab2.add(pair[0])
                bigrams2.add(pair)
                freq2[pair] = 1+freq2.get(pair, 0)
    freq_per_char2 = {i : dict() for i in vocab2}
    for bigram, f in freq2.items():
        i, j = bigram
        freq_per_char2[i][j] = f
    

new_token=('s', 'e')
new_token=('a', 's')
new_token=('as', 'se')
new_token=('asse', '$')
new_token=('s', 't')
new_token=('r', 'asse$')
new_token=('st', 'rasse$')
new_token=('e', 'r')
new_token=('e', 'n')
new_token=('g', '$')
new_token=('e', 'g$')
new_token=('c', 'h')


In [139]:
for _ in range(100):
    gen = ["$"]
    for _ in range(30):
        c = gen[-1]
        samples = list(freq_per_char2[c].keys())
        p_samples = list(freq_per_char2[c].values())
        next_c = choices(samples, p_samples)[0]
        gen.append(next_c)
        if next_c[-1]=="$":
            break
    print("".join(gen))

$Basengasse$
$Glzewanstweg$
$Frtzpfttrehnrs$
$Imatelachihl-Weg$
$Zägeglttierstrasse$
$Katenndrstrasse$
$Sttstrasse$
$Röngstrasse$
$Tusarühannastägstrasse$
$Grühneberge-Wanstrasse$
$Stenbalstrasse$
$Grauldsstrasse$
$Amangerstrasse$
$Wyertteikhauberste-Hergerstrasse$
$Imesenng$
$Wyernnbelenhuelttrasse$
$Dühaliweinchstrasse$
$Vihaderschösteserenrinweg$
$Basthof$
$Rugerm dstrasse$
$Rilerngnstrasse$
$Mer$
$Oelacklirsardmgstrasse$
$Fackolelenstrasse$
$Zweg$
$Latzweg$
$Kldlergasteitivassrubaumparobrünstrasse$
$Kinweg$
$Sig$
$Sphlgenbageggstrasse$
$Sirühldasipstrasse$
$Rantsterstrasse$
$Imenrstrasse$
$Prchöchen$
$Hue-Siweg$
$Imelstrasse$
$Imarhrartz$
$Boner$
$Bishoppldwatstrasse$
$Juchiestrasse$
$Holaugasse$
$Chütiternginhauchenusenstrasse$
$Entstrasse$
$Hotenstrasse$
$St-Frachtauenweg$
$Klhamlfansberstrasse$
$Ahschenstalifer Mühlzenstrasse$
$Klfitastttstrasse$
$Gif$
$Zonnorstrasse$
$Thlinenwindmergweg$
$Sesenstemischistrasse$
$Luerstrasse$
$Durie$
$Hiderwiwieggasse$
$Rikergstrasse$
$Schstrass

In [76]:
names[0:6]

['$Aargauerstrasse$',
 '$Abeggweg$',
 '$Abendweg$',
 '$Ackermannstrasse$',
 '$Ackersteinstrasse$',
 '$Ackerstrasse$']

In [75]:
names2[0:6]

['$Aargauerstrasse$',
 '$Abeggweg$',
 '$Abendweg$',
 '$Ackermannstrasse$',
 '$Ackersteinstrasse$',
 ['$', 'W', 'ü', 'h', 'r', 'e', '$']]

### How good is our model?

In [ ]:
# compute negative log likelihood of above model
# use NN to learn parameters (freq dictionary) to minimize log likelihood